In [1]:
ColabNotebook = 'google.colab' in str(get_ipython())

if ColabNotebook:
    # monta G-drive en entorno COLAB
    from google.colab import drive
    drive.mount('/content/drive/')

    # carpeta donde se encuentran archivos .py auxiliares
    FUENTES_DIR = '/content/drive/MyDrive/Colab Notebooks/FUENTES/'
    DATOS_DIR = '/content/drive/MyDrive/Colab Notebooks/DATOS/'      # carpeta donde se encuentran los datasets
else:
    # configuración para notebook con instalación LOCAL
    FUENTES_DIR = '../Fuentes'         # carpeta donde se encuentran archivos .py auxiliares
    DATOS_DIR   = '../Datos/' # carpeta donde se encuentran los datasets

# agrega ruta de busqueda donde tenemos archivos .py
import sys
sys.path.append(FUENTES_DIR)

In [2]:
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

import numpy as np
from sklearn import preprocessing, metrics, model_selection 
import pandas as pd

In [3]:
df = pd.read_csv(DATOS_DIR + 'Iris.csv')

# Tomamos todas las columnas menos la última
X = np.array(df.iloc[:, 0:-1])
Y = np.array(df.iloc[:,-1])

#Hay 50 muestras de cada tipo de flor
print(Y[[1,50,100]])

['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']


In [4]:
# convertimos las etiquetas categóricas en numéricas
encoder = preprocessing.LabelEncoder()
Y = encoder.fit_transform(Y)
print(Y[[1,50,100]])

[0 1 2]


In [5]:
#--- CONJUNTOS DE ENTRENAMIENTO Y TESTEO ---
X_train, X_test, Y_train, Y_test = model_selection.train_test_split( \
        X, Y, test_size=0.30) #, random_state=42)

Y_trainB = to_categorical(Y_train)
print(Y_trainB[:5, :])

[[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]]


In [6]:
normalizarEntrada = 1  # 1 si normaliza; 0 si no

if normalizarEntrada:
    # Escala los valores entre 0 y 1
    min_max_scaler = preprocessing.StandardScaler()
    X_train = min_max_scaler.fit_transform(X_train)
    X_test = min_max_scaler.transform(X_test)

### Definición del modelo

In [7]:
# Crear un modelo de capas secuenciales
model=Sequential()

# Agregar las capas al modelo
model.add(Dense(2, input_shape=[4], activation='LeakyReLU'))    #'ReLU', LeakyReLU'
model.add(Dense(3, activation='softmax'))

# Imprimir un resumen del modelo
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2)                 10        
                                                                 
 dense_1 (Dense)             (None, 3)                 9         
                                                                 
Total params: 19
Trainable params: 19
Non-trainable params: 0
_________________________________________________________________


In [8]:
# Configuración para entrenamiento
#model.compile(optimizer=SGD(learning_rate=0.05), loss='mse', metrics='accuracy')
model.compile(optimizer=SGD(learning_rate=0.05), loss='categorical_crossentropy', metrics='accuracy')

In [9]:
from tensorflow.keras.callbacks import LambdaCallback

# Define el callback
def on_epoch_end(epoch, logs):
    if (epoch + 1) % 100 == 0:  # Cambia 10 por el número de épocas deseado
        print(f"Epoch {epoch + 1}: loss = {logs['loss']}, accuracy = {logs['accuracy']}")

# Crea el LambdaCallback
epoch_end_callback = LambdaCallback(on_epoch_end=on_epoch_end)

# Entrena el modelo con el callback
model.fit(X_train,Y_trainB, epochs=1000, callbacks=[epoch_end_callback], verbose=0)

Epoch 100: loss = 0.12886466085910797, accuracy = 0.9714285731315613
Epoch 200: loss = 0.07507823407649994, accuracy = 0.961904764175415
Epoch 300: loss = 0.058176975697278976, accuracy = 0.9714285731315613
Epoch 400: loss = 0.05031540244817734, accuracy = 0.9714285731315613
Epoch 500: loss = 0.04561284929513931, accuracy = 0.9809523820877075
Epoch 600: loss = 0.04311607778072357, accuracy = 0.9714285731315613
Epoch 700: loss = 0.04039682075381279, accuracy = 0.9809523820877075
Epoch 800: loss = 0.03882600739598274, accuracy = 0.9714285731315613
Epoch 900: loss = 0.03770894557237625, accuracy = 0.9714285731315613
Epoch 1000: loss = 0.036170605570077896, accuracy = 0.9714285731315613


In [10]:
# Calcular el error del modelo
score = model.evaluate(X_train, Y_trainB) 
print('Error :', score[0]) 
print('Accuracy:', score[1])

4/4 [==============================] - 0s 2ms/step - loss: 0.0358 - accuracy: 0.9714
Error : 0.035751793533563614
Accuracy: 0.9714285731315613


In [11]:
# predecir la salida del modelo
Y_pred = model.predict(X_train)
print(Y_pred[:5])

Y_pred2 = np.argmax(Y_pred,axis=1)
print(Y_pred2[:5])

print("%% aciertos X_train : %.3f" % metrics.accuracy_score(Y_train, Y_pred2))

4/4 [==============================] - 0s 6ms/step
[[4.51713195e-03 9.92592752e-01 2.89017451e-03]
 [9.16678437e-06 1.03224724e-04 9.99887526e-01]
 [9.99993742e-01 6.22506559e-06 2.28215252e-10]
 [9.99983966e-01 1.59620649e-05 8.01237965e-10]
 [3.10441968e-03 9.87047493e-01 9.84807312e-03]]
[1 2 0 0 1]
% aciertos X_train : 0.971


In [12]:
report = metrics.classification_report(Y_train, Y_pred2)
print("Confusion matrix Training:\n%s" % report) 

MM = metrics.confusion_matrix(Y_train, Y_pred2)
print("Confusion matrix:\n%s" % MM)

Confusion matrix Training:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        35
           1       0.97      0.94      0.95        33
           2       0.95      0.97      0.96        37

    accuracy                           0.97       105
   macro avg       0.97      0.97      0.97       105
weighted avg       0.97      0.97      0.97       105

Confusion matrix:
[[35  0  0]
 [ 0 31  2]
 [ 0  1 36]]


In [13]:
model.save("miniIris.h5")

In [14]:
from keras.models import load_model
modelo2 = load_model("miniIris.h5")
modelo2.summary()

# predecir la salida del modelo
Y_pred = modelo2.predict(X_train)
print(Y_pred[:5])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2)                 10        
                                                                 
 dense_1 (Dense)             (None, 3)                 9         
                                                                 
Total params: 19
Trainable params: 19
Non-trainable params: 0
_________________________________________________________________
4/4 [==============================] - 0s 3ms/step
[[4.51713195e-03 9.92592752e-01 2.89017451e-03]
 [9.16678437e-06 1.03224724e-04 9.99887526e-01]
 [9.99993742e-01 6.22506559e-06 2.28215252e-10]
 [9.99983966e-01 1.59620649e-05 8.01237965e-10]
 [3.10441968e-03 9.87047493e-01 9.84807312e-03]]
